In [1]:
import sys

sys.argv = [""]
del sys

import argparse
import warnings
warnings.filterwarnings('ignore')

In [2]:
def create_cfg():
    parser = argparse.ArgumentParser()
    for key, val in args.items():
        parser.add_argument(key, default=val)
        # print("--" + key, val)
    cfg = parser.parse_args()
    return cfg

In [3]:
args = {}

# Data args
# args["--train_ds_dir"] = '../../../../Datasets/Places365_Standard/data_large'
# args["--val_ds_dir"] = '../../../../Datasets/Places365_Standard/val_large'
args["--train_ds_dir"] = '../samples/Places365'
args["--val_ds_dir"] = '../samples/Places365'

# args["--CKPT_DIR"] = '../../ckpts/exp1'
# args["--LOG_DIR"] = '../../logs/exp1'
# args["--SAMPLE_DIR"] = '../../samples/exp1'

args["--CKPT_DIR"] = '../../../experiments/inpaint/ckpts/exp1'
args["--LOG_DIR"] = '../../../experiments/inpaint/logs/exp1'
args["--SAMPLE_DIR"] = '../../../experiments/inpaint/samples/exp1'

# Image and Mask args
args["--crop_size"] = (384, 384)
args["--mask_type"] = 'free_form'
args["--mask_num"] = 20
args["--max_angle"] = 10 
args["--max_len"] = 40
args["--max_width"] = 50 
args["--margin"] = (10, 10) 
args["--bbox_shape"] = (100, 100) 

# Training args
args["--epochs"] = 1
args["--batch_size"] =  2
args["--num_workers"] = 1
args["--device_id"] = 0
args["--lr_g"] =  1e-4 
args["--lr_d"] = 1e-4 
args["--b1"] = 0.5
args["--b2"] = 0.999
args["--weight_decay"] = 0
args["--lambda_l1"] = 10
args["--lambda_perceptual"] = 10
args["--lambda_gan"] = 1 
args["--lr_decrease_epoch"] = 10
args["--lr_decrease_factor"] = 0.5 

args["--LOG_INTERVAL"] = 100
args["--SAVE_SAMPLES_INTERVAL"] = 5
args["--SAVE_SAMPLE_COUNT"] = 5

args["--use_cuda"] = True


# Model args
args["--in_channels"] = 4
args["--out_channels"] = 3
args["--latent_channels"] = 64
args["--pad_type"] = "zero"
args["--activation"] = "elu"
args["--norm_d"] = "none"
args["--norm_g"] = "none"
args["--init_type"] = "kaiming"
args["--init_gain"] = 0.02
args["--use_perceptualnet"] = True

In [4]:
cfg = create_cfg()

In [5]:
from inpaint.data import PlacesDataset
from torch.utils.data import DataLoader
from inpaint.core.discriminator import PatchDiscriminator
from inpaint.core.generator import GatedGenerator
from inpaint.tools import Trainer

In [6]:
train_ds = PlacesDataset(path_dir = cfg.train_ds_dir, transform_config=('to_tensor', 'random_crop', 'norm'), crop_size=(384,384))
print(f"Total training images: {len(train_ds)}")

Total training images: 24


In [7]:
val_ds = PlacesDataset(path_dir = cfg.val_ds_dir, transform_config=('to_tensor', 'center_crop', 'norm'), crop_size=(384,384))
print(f"Total validation images: {len(val_ds)}")

Total validation images: 24


In [8]:
train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, pin_memory=True, num_workers=cfg.num_workers)

In [9]:
val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=True, pin_memory=True, num_workers=cfg.num_workers)

In [10]:
generator = GatedGenerator(cfg)
discriminator = PatchDiscriminator(cfg)

In [11]:
trainer = Trainer(
    cfg=cfg,
    discriminator=discriminator,
    generator=generator,
    train_loader=train_loader,
    val_loader=val_loader
)

Running on CUDA device: 0


In [12]:
trainer.train()

Epoch 1 of 1
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Iteration 0/0 Discriminator Loss: 1.000274419784546, GAN Loss: 0.01302043441683054, Reconstruction Loss: 9.977224349975586, Overall Generator Loss: 204.37979125976562

Validation Loss:
 Discriminator Loss: 0.838935653368632, GAN Loss: 0.300351832062006, Reconstruction Loss: 8.397455175717672, Overall Generator Loss: 151.44961547851562
New avg validation loss discriminator!
New avg validation loss generator!
New avg validation loss reconstruction!
New avg validation loss overall!
Saved best generator and discriminator


Saved final best generator and discriminator
